In [1]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import colors
from evaluation import trilateration_centroid, trilateration_centroid_vectorization
from visualization import draw_centroids
from torchvision.transforms import Normalize
import cv2
from conventional_centroiding.threshold_method import centroid_com, detection_globalThreshold, centroiding_CenterOfMass, detection_ST16, centroiding_GaussianGrid, detection_WITM, detection_erosion_dilation

In [2]:
device = torch.device("cuda:0")
pixel_size = 6/1000
radius = 7

img_np = np.load('nightsky_june16_2021.npy' )
#img_np = cv2.imread('skycam/skycam_1.png', cv2.IMREAD_GRAYSCALE)
center = img_np.shape 
h = 480
w = 640
x = int(center[1]/2 - w/2)
y = int(center[0]/2 - h/2) 
img_np = img_np[y:y+h, x:x+w]

# # for data_straylight
# mean = [44.1619381]
# std =  [60.98225565]

# for data_Oct19
mean = [25.36114133]
std =  [44.31162568]

img = torch.from_numpy(img_np).float()
img = img.to(device)
img = img.unsqueeze(0)
img = img.unsqueeze(0)

# get predictions
#model = torch.load("./saved_models/ELUNet_inter_1.pt")
model = torch.load("./saved_models/ELUNet_inter_2_90.pt")
model.eval()

prediction = model(Normalize(mean, std)(img)) # only normalize images for neural network so that conventional centroid can use raw images
seg_prediction = 1.0 * (torch.sigmoid(prediction[0][0]) > 0.5)
seg_prediction = seg_prediction.detach().cpu().numpy()
dist_prediction = prediction[0][1]
dist_prediction = dist_prediction.detach().cpu().numpy()

img = img[0][0].detach().cpu().numpy()

In [3]:
# get estimated centroids: machine learning 

centroid_est_1 = trilateration_centroid(dist_prediction.copy(), seg_prediction.copy(), radius, pixel_size)

centroid_est_2 = trilateration_centroid_vectorization(dist_prediction.copy(), seg_prediction.copy(), radius, pixel_size)




In [4]:
%timeit -o -r 10 trilateration_centroid(dist_prediction.copy(), seg_prediction.copy(), radius, pixel_size)

116 ms ± 2.34 ms per loop (mean ± std. dev. of 10 runs, 10 loops each)


<TimeitResult : 116 ms ± 2.34 ms per loop (mean ± std. dev. of 10 runs, 10 loops each)>

In [5]:
%timeit -o -r 10 trilateration_centroid_vectorization(dist_prediction.copy(), seg_prediction.copy(), radius, pixel_size)

4.49 ms ± 131 µs per loop (mean ± std. dev. of 10 runs, 100 loops each)


<TimeitResult : 4.49 ms ± 131 µs per loop (mean ± std. dev. of 10 runs, 100 loops each)>